In [1]:
!pip install -q opencv-python dlib imutils tensorflow
!apt-get install -y cmake
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
--2025-05-24 14:33:29--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-05-24 14:33:29--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  39.0MB/s    in 1.6s    

2025-05-24 14:33:31 (39.0 MB/s) - ‘shape_predictor_68_face_la

In [2]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ankithka","key":"84daa5f8fb4a5b18f5400ec216c015a9"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d akashshingha850/mrl-eye-dataset
!unzip -q mrl-eye-dataset.zip -d /content/MRL_eye_dataset


Dataset URL: https://www.kaggle.com/datasets/akashshingha850/mrl-eye-dataset
License(s): MIT
 78% 256M/329M [00:00<00:00, 776MB/s] 
100% 329M/329M [00:00<00:00, 767MB/s]


In [5]:
!kaggle datasets download -d davidvazquezcic/yawn-dataset
!unzip -q yawn-dataset.zip -d /content/Yawn_Dataset


Dataset URL: https://www.kaggle.com/datasets/davidvazquezcic/yawn-dataset
License(s): CC-BY-NC-SA-4.0
  0% 0.00/16.9M [00:00<?, ?B/s]
100% 16.9M/16.9M [00:00<00:00, 953MB/s]


In [6]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

def load_images(folder, label):
    data = []
    for filename in glob(os.path.join(folder, '*')):
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (50, 50))
        img = img / 255.0
        data.append((img, label))
    return data

# Load MRL Eye Dataset
eye_open = load_images('/content/MRL_eye_dataset/data/test/awake', 0)
eye_closed = load_images('/content/MRL_eye_dataset/data/test/sleepy', 1)

# Load Yawn Dataset
yawn = load_images('/content/Yawn_Dataset/yawn', 1)
no_yawn = load_images('/content/Yawn_Dataset/no_yawn', 0)

eye_data = eye_open + eye_closed
mouth_data = yawn + no_yawn

def prepare_dataset(data):
    X = np.array([img.reshape(50, 50, 1) for img, _ in data])
    y = np.array([label for _, label in data])
    return train_test_split(X, y, test_size=0.2, random_state=42)

X_eye_train, X_eye_test, y_eye_train, y_eye_test = prepare_dataset(eye_data)
X_mouth_train, X_mouth_test, y_mouth_train, y_mouth_test = prepare_dataset(mouth_data)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def build_cnn(conv_layers):
    model = Sequential()
    model.add(Conv2D(conv_layers[0], (3,3), activation='relu', input_shape=(50,50,1)))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    for filters in conv_layers[1:]:
        model.add(Conv2D(filters, (3,3), activation='relu'))
        model.add(MaxPooling2D(2,2))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [8]:
# Build and train eye models
eye_cnn1 = build_cnn([32, 64])
eye_cnn2 = build_cnn([32, 64, 128])
eye_cnn3 = build_cnn([64, 128, 256])

eye_cnn1.fit(X_eye_train, y_eye_train, epochs=10, validation_split=0.1, batch_size=64)
eye_cnn2.fit(X_eye_train, y_eye_train, epochs=10, validation_split=0.1, batch_size=64)
eye_cnn3.fit(X_eye_train, y_eye_train, epochs=10, validation_split=0.1, batch_size=64)

# Build and train mouth models
mouth_cnn1 = build_cnn([32, 64])
mouth_cnn2 = build_cnn([32, 64, 128])
mouth_cnn3 = build_cnn([64, 128, 256])

mouth_cnn1.fit(X_mouth_train, y_mouth_train, epochs=10, validation_split=0.1, batch_size=64)
mouth_cnn2.fit(X_mouth_train, y_mouth_train, epochs=10, validation_split=0.1, batch_size=64)
mouth_cnn3.fit(X_mouth_train, y_mouth_train, epochs=10, validation_split=0.1, batch_size=64)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.8392 - loss: 0.5293 - val_accuracy: 0.4753 - val_loss: 1.3718
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9281 - loss: 0.1986 - val_accuracy: 0.5335 - val_loss: 1.6465
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9471 - loss: 0.1500 - val_accuracy: 0.8300 - val_loss: 0.3608
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9519 - loss: 0.1286 - val_accuracy: 0.9301 - val_loss: 0.1888
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9590 - loss: 0.1097 - val_accuracy: 0.9338 - val_loss: 0.1912
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9664 - loss: 0.0967 - val_accuracy: 0.9713 - val_loss: 0.0953
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9718 - loss: 0.0770 - val_accuracy: 0.9029 - val_loss: 0.2553
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9707 - loss: 0.0781 - val_accuracy: 

In [9]:
def ensemble_predict(models, X):
    preds = [model.predict(X, verbose=0) for model in models]
    avg_preds = np.mean(preds, axis=0)
    return (avg_preds > 0.5).astype(int)

# Evaluate Eye and Mouth Ensemble
from sklearn.metrics import accuracy_score

eye_models = [eye_cnn1, eye_cnn2, eye_cnn3]
mouth_models = [mouth_cnn1, mouth_cnn2, mouth_cnn3]

eye_pred = ensemble_predict(eye_models, X_eye_test)
mouth_pred = ensemble_predict(mouth_models, X_mouth_test)

print("Eye Ensemble Accuracy:", accuracy_score(y_eye_test, eye_pred))
print("Mouth Ensemble Accuracy:", accuracy_score(y_mouth_test, mouth_pred))


Eye Ensemble Accuracy: 0.9814542243155726
Mouth Ensemble Accuracy: 0.8466796875


In [10]:
import dlib
from imutils import face_utils
from tensorflow.keras.preprocessing.image import img_to_array

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def detect_drowsiness(image, eye_models, mouth_models):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = face_detector(gray, 0)

    for rect in rects:
        shape = landmark_predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        left_eye = shape[36:42]
        right_eye = shape[42:48]
        mouth = shape[48:68]

        def crop_and_process(region):
            x, y, w, h = cv2.boundingRect(np.array([region]))
            roi = gray[y:y + h, x:x + w]
            roi = cv2.resize(roi, (50, 50))
            roi = roi.astype("float32") / 255.0
            return roi.reshape(1, 50, 50, 1)

        left_eye_img = crop_and_process(left_eye)
        right_eye_img = crop_and_process(right_eye)
        mouth_img = crop_and_process(mouth)

        eye_pred = (ensemble_predict(eye_models, left_eye_img)[0] + ensemble_predict(eye_models, right_eye_img)[0]) / 2
        mouth_pred = ensemble_predict(mouth_models, mouth_img)[0]

        if eye_pred == 1 or mouth_pred == 1:
            print("⚠️ Drowsiness Detected!")
        else:
            print("✅ Driver is Alert")


In [11]:
frame = cv2.imread('/content/drive/MyDrive/Colab Notebooks/yawn-test.jpeg')  # Upload your test image
detect_drowsiness(frame, eye_models, mouth_models)


⚠️ Drowsiness Detected!


In [12]:
from google.colab import files
files.download("eye_cnn1.h5")
files.download("eye_cnn2.h5")
files.download("eye_cnn3.h5")

files.download("mouth_cnn1.h5")
files.download("mouth_cnn2.h5")
files.download("mouth_cnn3.h5")


FileNotFoundError: Cannot find file: eye_cnn1.h5